In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

2022-01-17 20:51:49.872238: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
is_cuda_gpu_min_3

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-01-17 20:51:51.058926: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-17 20:51:51.060715: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-17 20:51:51.095899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 20:51:51.096110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 1

True

atform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-01-17 20:51:51.099774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-01-17 20:51:51.100073: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-01-17 20:51:51.101708: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-01-17 20:51:51.102767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.10
2022-01-17 20:51:51.107025: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-17 20:51:51.107171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, 

In [3]:
train = np.load('../data/WN18_numpy/train.npy')
validation = np.load('../data/WN18_numpy/validation.npy')
test = np.load('../data/WN18_numpy/test.npy')

entities = np.load('../data/WN18_numpy/entities.npy')
relations = np.load('../data/WN18_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int32)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int32)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int32)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2022-01-17 20:51:51.881545: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-17 20:51:51.881821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 20:51:51.882149: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-01-17 20:51:51.882229: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-17 20:51:51.882302: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-17 20:51:51.882333: I tensorflow/stream_executor/platfor

In [4]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.0001, dtype = tf.float64)
beta = tf.Variable(0.00001, dtype = tf.float64)
z = tf.constant(0.5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn2')      

In [5]:
# @tf.function
def calculateRank(triple):  
    start = time.time() 
    selectedEntities = tf.cast(tf.gather(entities, 0, axis=1), dtype=tf.int32)
    _head_index = tf.gather(triple,0)
    _tail_index = tf.gather(triple,1)
    _relation_index = tf.gather(triple,2)  
    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 1, axis=1) == _tail_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 1, axis=1) == _tail_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 1, axis=1) == _tail_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))
    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedH = tf.concat([  tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _tail_index) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedH = tf.concat([couraptedH, tf.reshape(triple, (1,3))], 0)
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    div = tf.constant(3)
    sliceSize = tf.cast(tf.math.floor(tf.math.truediv(couraptedH.shape[0], div)),dtype=tf.int32)
    s0, s1, s2= tf.split(couraptedH, num_or_size_splits=[
        sliceSize , 
        sliceSize , 
        tf.math.subtract(couraptedH.shape[0] , tf.math.multiply(sliceSize, 2)) ], axis=0)
    couraptedH = None
    p0 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s0,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s0,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p1 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s1,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s1,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p2 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s2,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s2,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p = tf.concat([p0,p1,p2,], 0)
    rankH = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    
    p0 = None
    p1 = None
    p2 = None
    p = None

    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 0, axis=1) == _head_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 0, axis=1) == _head_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 0, axis=1) == _head_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))

    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedT = tf.concat([   tf.fill([selectedEntitiesFinal.shape[0], 1], _head_index) , tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedT = tf.concat([couraptedT, tf.reshape(triple, (1,3))], 0)    
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    div = tf.constant(3)
    sliceSize = tf.cast(tf.math.floor(tf.math.truediv(couraptedT.shape[0], div)),dtype=tf.int32)
    s0, s1, s2= tf.split(couraptedT, num_or_size_splits=[
        sliceSize , 
        sliceSize , 
        tf.math.subtract(couraptedT.shape[0] , tf.math.multiply(sliceSize, 2)) ], axis=0)
    couraptedT = None
    p0 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s0,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s0,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p1 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s1,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s1,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p2 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s2,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s2,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p = tf.concat([p0,p1,p2,], 0)
    rankT = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    p = None
    print(rankH, rankT)
    if tf.math.greater(rankH , rankT):
        return rankT
    else:
        return rankH
    

In [6]:
def mrr(ranks):
    inverse = []
    one = tf.constant(1.0, dtype = tf.float64)
    for rank in ranks:
        inverse.append( tf.math.truediv(one , tf.cast(rank,tf.float64)))
    summ = tf.reduce_sum(inverse)
    return tf.math.multiply(tf.math.truediv(one , len(inverse)) , summ)

In [7]:
start = time.time()
paddings = tf.constant([[0, 1,], [0, 0]])
t_row = 500 #train.shape[0]
for x in range(0 , iteration):
    tf.random.shuffle(train, seed=None, name=None)
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        # print(samples)
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        # print(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        # print(_sigmoid)
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        # print(cost)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        # print(g,g1)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (6, 1))) , _nn1_samples)
        # print(_nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)

        _nn2_sample = tf.math.add_n([tf.math.reduce_sum(tf.math.multiply(_vRel , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel, tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False)])
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))
  
        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(_vHead , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _nn2_sample)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    
    print(time.time() - start) 
    # print(nn2)       
    if x % 3 == 0 and x > 0:
        tensor = []
        for tIndex in range(0 , 100):
            
            triple = tf.gather(validation,tIndex)
            print(triple)
            tensor.append(calculateRank(triple))
        print(mrr(tensor))
        print(time.time() - start)
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 

2022-01-17 20:51:52.207066: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


6.469274520874023
11.488004684448242
16.273183345794678
21.42380666732788
tf.Tensor([12344 12360     3], shape=(3,), dtype=int32)
tf.Tensor([[40906]], shape=(1, 1), dtype=int64) tf.Tensor([[32063]], shape=(1, 1), dtype=int64)
tf.Tensor([20598 13233     1], shape=(3,), dtype=int32)
tf.Tensor([[40942]], shape=(1, 1), dtype=int64) tf.Tensor([[36960]], shape=(1, 1), dtype=int64)
tf.Tensor([27017 26682    15], shape=(3,), dtype=int32)


2022-01-17 20:52:21.639840: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at scatter_nd_op.cc:130 : Invalid argument: indices[7] = [-26532] does not index into shape [40943]


InvalidArgumentError: indices[7] = [-26532] does not index into shape [40943] [Op:ScatterNd]

In [ ]:
# tensor = []
# for tIndex in range(0 , 15):
#     triple = tf.gather(validation,tIndex)
#     tensor.append(calculateRank(triple))
# print(mrr(tensor))
# tensor = tf.zeros(validation.shape[0], tf.int64)

# c = lambda i: tf.less(i, validation.shape[0])
# i = tf.constant(0)
# b = lambda i: (calculateRank,tf.gather(validation,i),)
# # print(tf.gather(validation,i))
# r = tf.while_loop(c, b, loop_vars= [i])
# print(tensor)
x = tf.Variable(tf.random.uniform([18, 3], -1, 1))
div = tf.constant(3)
sliceSize = tf.cast(tf.math.floor(tf.math.truediv(x.shape[0], div)),dtype=tf.int32)
s0, s1, s2= tf.split(x, num_or_size_splits=[
    sliceSize , 
    sliceSize , 
    tf.math.subtract(x.shape[0] , tf.math.multiply(sliceSize, 2)) ], axis=0)
s0